# Лабораторная работа №2 (Проведение исследований с логистической и линейной регрессией)

## Создание бейзлайна и оценка качества

### Обучение моделей из sklearn для выбранных наборов данных

#### Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from scipy.spatial import distance
import numpy as np
from collections import Counter

#### Подготовка данных

##### Классификация (Credit Card Fraud Detection Dataset 2023)

**Загрузка данных:**

In [2]:
fraud_data = pd.read_csv('/kaggle/input/credit-card-fraud-detection-dataset-2023/creditcard_2023.csv')
fraud_data = fraud_data.sample(frac=0.05)
X_fraud = fraud_data.iloc[:, :-1]  # Признаки
y_fraud = fraud_data.iloc[:, -1]   # Метки классов

**Разделение на тренировочный и тестовый наборы:**

In [3]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

**Замена пропущенных значений**

In [4]:
def simple_imputer_df(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return pd.DataFrame(imputer.fit_transform(x_train)), pd.DataFrame(imputer.transform(x_test))

In [5]:
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)

**Масштабирование данных**

In [6]:
def scaling(x_train, x_test):
    scaler = StandardScaler()
    return scaler.fit_transform(x_train), scaler.transform(x_test)

In [7]:
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

##### Регрессия (Gold Price Regression)

**Загрузка данных:**

In [8]:
gold_data = pd.read_csv('/kaggle/input/financial-data/financial_regression.csv')

X_gold = gold_data.iloc[:, 1:-1]  # Признаки
y_gold = gold_data.iloc[:, -1]   # Целевая переменная

**Разделение на тренировочный и тестовый наборы:**

In [9]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

**Замена пропущенных значений:**

In [10]:
def simple_imputer(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return imputer.fit_transform(x_train), imputer.transform(x_test)

In [11]:
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)

**Удаление строк с пропущенными значениями:**

In [12]:
def delete_none(x_train, y_train, x_test, y_test):
    mask = ~np.isnan(y_train)
    mask_test = ~np.isnan(y_test)
    return x_train[mask], y_train[mask], x_test[mask_test], y_test[mask_test]

In [13]:
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

#### Обучение моделей

##### Классификация

In [14]:
log_reg = LogisticRegression()
log_reg.fit(X_train_fraud, y_train_fraud)

LogisticRegression()

##### Регрессия

In [15]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_gold, y_train_gold)

LinearRegression()

### Оценка качества моделей по выбранным метрикам

##### Классификация

In [16]:
def quality_eval_classification(X_test, y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    return accuracy, f1

In [17]:
accuracy_origin, f1_origin = quality_eval_classification(X_test_fraud, y_test_fraud, log_reg)

Accuracy: 0.9973624054861966
F1 Score: 0.9973623731891539


##### Регрессия

In [18]:
def quality_eval_regression(X_test, y_test, model):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
    return mae, r2

In [19]:
mae_origin, r2_origin = quality_eval_regression(X_test_gold, y_test_gold, lin_reg)

Mean Absolute Error: 2502444.4998075487
R-squared: 0.6204675813233773


## Улучшение бейзлайна

### Гипотезы

1. Подбор гиперпараметров:
   - Использование Grid Search с кросс-валидацией для оптимизации модели.

### Проверка гипотез

#### 1. Подбор гиперпараметров

##### Классификация

Подготовка данных:

In [20]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

Определение параметров для перебора:

In [21]:
param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear', 'saga']
}

Настройка и обучение GridSearchCV:

In [22]:
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_fraud, y_train_fraud)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.1, 1, 10],
                         'solver': ['lbfgs', 'liblinear', 'saga']})

Вывод лучших параметров и результата:

In [23]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'C': 10, 'solver': 'lbfgs'}
Best cross-validation accuracy:  0.9982853374367993


Оценка качества модели:

In [24]:
quality_eval_classification(X_test_fraud, y_test_fraud, grid_search.best_estimator_)

Accuracy: 0.9982416036574644
F1 Score: 0.998241590282027


(0.9982416036574644, 0.998241590282027)

##### Регрессия

Подготовка данных:

In [25]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

Определение параметров для перебора:

In [26]:
param_grid = {
    'alpha': np.logspace(-4, 4, 50)
}

Настройка и обучение GridSearchCV:

In [27]:
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_gold, y_train_gold)

GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095476e-04, 1.38949549e-03,
       2.02358965e-03, 2.94705170e-03, 4.29193426e-03, 6.25055193e-03,
       9.10298178e-03, 1.32571137e-02, 1.93069773e-02, 2.81176870e-02,
       4.09491506e-02, 5.96362332e-02, 8.68511...
       3.72759372e+00, 5.42867544e+00, 7.90604321e+00, 1.15139540e+01,
       1.67683294e+01, 2.44205309e+01, 3.55648031e+01, 5.17947468e+01,
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04])},
             scoring='neg_mean_squared_error')

Вывод лучших параметров и результата:

In [28]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'alpha': 0.019306977288832496}
Best cross-validation accuracy:  -12138305057459.457


Оценка модели с лучшими параметрами на тестовом наборе:

In [29]:
quality_eval_regression(X_test_gold, y_test_gold, grid_search.best_estimator_)

Mean Absolute Error: 2499103.1013491563
R-squared: 0.6198824533246513


(2499103.1013491563, 0.6198824533246513)

### Формирование улучшенного бейзлайна

##### Классификация

Формирование улучшенного бейзлайна:

In [30]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)
                                   
log_reg = LogisticRegression(max_iter=1000, C=10, solver="liblinear")

##### Регрессия

In [31]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

lin_reg = Ridge(alpha=np.float64(0.019306977288832496))

### Обучение модели с улучшенным бейзлайном

##### Классификация

Обучение модели:

In [32]:
log_reg.fit(X_train_fraud, y_train_fraud)

LogisticRegression(C=10, max_iter=1000, solver='liblinear')

##### Регрессия

Обучение модели:

In [33]:
lin_reg.fit(X_train_gold, y_train_gold)

Ridge(alpha=0.019306977288832496)

### Оценка качества модели с улучшенным бейзлайном

##### Классификация

Оценка качества модели:

In [34]:
accuracy_improved, f1_improved = quality_eval_classification(X_test_fraud, y_test_fraud, log_reg)

Accuracy: 0.998417443291718
F1 Score: 0.99841742890493


##### Регрессия

Оценка качества модели:

In [35]:
mae_improved, r2_improved = quality_eval_regression(X_test_gold, y_test_gold, lin_reg)

Mean Absolute Error: 2499103.1013491563
R-squared: 0.6198824533246513


### Сравнение результатов первоначального бейзлайна с улучшенным бейзлайном

##### Классификация

Видим, что результаты немного улучшились:

In [36]:
print(f"Accuracy difference: {accuracy_improved-accuracy_origin}")
print(f"F1 Score difference: {f1_improved-f1_origin}")

Accuracy difference: 0.0010550378055214082
F1 Score difference: 0.001055055715776132


##### Регрессия

Видим, что результаты немного улучшились:

In [37]:
print(f"Mean Absolute Error difference: {mae_origin-mae_improved}")
print(f"R-squared difference: {r2_improved-r2_origin}")

Mean Absolute Error difference: 3341.3984583923593
R-squared difference: -0.0005851279987260094


### Выводы

С улучшенным бейзлайном качество модели совсем немного возрасло, поэтому можно сделать следующие выводы:
1. Хорошая начальная настройка: Модель уже была близка к оптимальной конфигурации.
2. Ограниченное влияние: Гиперпараметры имеют малое влияние на производительность в контексте ваших данных.
3. Фокус на данных: Возможно, лучшее улучшение можно получить, работая с данными — очисткой, добавлением новых признаков и так далее.
4. Переобучение: Небольшое улучшение также может быть признаком наклона к переобучению, особенно если качество на тестовом наборе данных не улучшилось.
5. Иные методы: Есть возможность попробовать другие модели или ансамблевые методы для улучшения качества.

## Имплементация алгоритма машинного обучения

### Самостоятельная имплементация

##### Классификация

Логистическая регрессия - алгоритм классификации, который использует сигмоидальную функцию для преобразования линейной комбинации признаков в вероятность класса.

In [38]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iter=1000):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iter):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

##### Регрессия

Линейная регрессия - это метод, который моделирует зависимость между входными признаками и целевым значением с использованием линейной зависимости.

In [39]:
class MyLinearRegression:
    def __init__(self, learning_rate=0.01, n_iter=1000):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iter):
            y_predicted = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

### Обучение имплементированной модели

##### Классификация

Подготовка данных:

Преобразуем строковые метки в числовые

In [40]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_fraud)

In [41]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_encoded, test_size=0.2, random_state=42)
X_train_fraud, X_test_fraud = simple_imputer_df(X_train_fraud, X_test_fraud)
X_train_fraud, X_test_fraud = scaling(X_train_fraud, X_test_fraud)

log_reg = MyLogisticRegression()

Обучение модели:

In [42]:
log_reg.fit(X_train_fraud, y_train_fraud)

##### Регрессия

Подготовка данных:

In [43]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

lin_reg = MyLinearRegression()

Обучение модели:

In [44]:
lin_reg.fit(X_train_gold, y_train_gold)

### Оценка качества имплементированных моделей

##### Классификация

Оценка качества модели:

In [45]:
accuracy_my, f1_my = quality_eval_classification(X_test_fraud, y_test_fraud, log_reg)

Accuracy: 0.9701072621768947
F1 Score: 0.9700777295604297


##### Регрессия

Оценка качества модели:

In [46]:
mae_my, r2_my = quality_eval_regression(X_test_gold, y_test_gold, lin_reg)

Mean Absolute Error: 2734922.748168548
R-squared: 0.5196339914918888


### Сравнение результатов первоначального бейзлайна и имплементированных моделей

##### Классификация

Видим, что результаты имплементированной модели хуже:

In [47]:
print(f"Accuracy difference: {accuracy_my-accuracy_origin}")
print(f"F1 Score difference: {f1_my-f1_origin}")

Accuracy difference: -0.0272551433093019
F1 Score difference: -0.02728464362872418


##### Регрессия

Видим, что результаты имплементированной модели хуже:

In [48]:
print(f"Mean Absolute Error difference: {mae_my-mae_origin}")
print(f"R-squared difference: {r2_my-r2_origin}")

Mean Absolute Error difference: 232478.2483609994
R-squared difference: -0.10083358983148849


### Выводы

Оценка качества наших имплементированных моделей оказалась хуже.
Можно сделать следующие выводы:
1. Предобработка данных важна: Масштабирование и нормализация данных могут значительно повлиять на качество модели.
2. Оптимизация гиперпараметров: Правильная настройка параметров, таких как темп обучения и количество итераций, необходима для достижения оптимальной производительности.
3. Регуляризация: Помогает избежать переобучения и может улучшить обобщающую способность модели.
4. Алгоритмическая точность: Правильная реализация алгоритма и внимание к численным ошибкам критичны для качества модели.
5. Используйте проверенные решения: Изучение и сравнение с библиотечными реализациями может помочь выявить слабые места вашей модели.
6. Тщательно изучайте данные: Соблюдение чистоты, отсутствие пропусков и ошибок в данных — важный аспект построения качественной модели.

### Добавление техник из улучшенного бейзлайна

##### Регрессия

Формирование улучшенного бейзлайна:

In [49]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

lin_reg = MyLinearRegression(learning_rate=np.float64(0.019306977288832496))

### Обучение на улучшенном бейзлайне

##### Регрессия

In [50]:
lin_reg.fit(X_train_gold, y_train_gold)

### Оценка качества моделей на улучшенном бейзлайне

##### Регрессия

In [51]:
mae_my_impoved, r2_my_impoved = quality_eval_regression(X_test_gold, y_test_gold, lin_reg)

Mean Absolute Error: 2736143.926621333
R-squared: 0.5192060277943524


### Сравнение результатов

##### Регрессия

In [52]:
print(f"Mean Absolute Error difference: {mae_my_impoved-mae_improved}")
print(f"R-squared difference: {r2_my_impoved-r2_improved}")

Mean Absolute Error difference: 237040.82527217688
R-squared difference: -0.10067642553029887


### Выводы

Эти методы не дали улучшение результата в нашем случае.